# Building fast queries on a csv

In this project we will explore time and space complexities of algorithms. 

The data we will use is a csv file with 1300 laptops from [Kaggle](https://www.kaggle.com/ionaskel/laptop-prices). The columns are:

* **ID**: A unique identifier for the laptop.
* **Company**: The name of the company that produces the laptop.
* **Product**: The name of the laptop.
* **TypeName**: The type of laptop.
* **Inches**: The size of the screen in inches.
* **ScreenResolution**: The resolution of the screen.
* **CPU**: The laptop CPU.
* **RAM**: The amount of RAM in the laptop.
* **Memory**: The size of the hard drive.
* **GPU**: The graphics card name.
* **OpSys**: The name of the operating system.
* **Weight**: The laptop weight.
* **Price**: The price of the laptop.

In [1]:
import csv
with open('laptops.csv') as f:
    laptops = list(csv.reader(f))
    header = laptops[0]
    rows = laptops[1:]
    
print(header)
print(rows[:5])

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB 

The **goal** of this project is to create a **class** that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:

* Given a laptop id, find the corresponding data.
* Given an amount of money, find whether there are two laptops whose total price is that given amount.
* Identify all laptops whose price falls within a given budget.

In [2]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = list(csv.reader(f))
            self.header = read_file[0]
            self.rows = read_file[1:]
        for row in self.rows:    
            row[-1] = int(row[-1])
        
laptops = Inventory('laptops.csv')
print(laptops.header)
print(len(laptops.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


### The class
Throughout this project, we will make several improvements to the `Inventory` class. We will create a new cell at the start of each screen and copy and paste the last version of the class and modify the later. This will help us to keep track of the changes.

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

For this, we will write a function named `get_laptop_from_id()`. This function will take as argument the identifier of the laptop and return the full row of the laptop with that id.

In [3]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = list(csv.reader(f))
            self.header = read_file[0]
            self.rows = read_file[1:]
        for row in self.rows:    
            row[-1] = int(row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
            else:
                return None
            
laptops = Inventory('laptops.csv')
print(laptops.get_laptop_from_id('6571244'))
print(laptops.get_laptop_from_id('3362736'))

['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
None


## Improving time complexity O(R)
The algorithm from the previous cell requires us to look at every single row to find the one that we are looking for (or decide that such a row does not exist). This algorithm has time complexity O(R) where R is the number of rows.

We can solve this problem more efficiently by preprocessing the data. Using a set, we can check in constant time whether a given identifier exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of a set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

In our dataset, we only have about 1,300 laptops, so it might seem unnecessary to improve the performance of this query. However, you have to imagine that this code could be used in situations where the inventory contains millions of rows. Also, if we perform a lot of queries, even on a small dataset, the slow query performance will start to add up. It might eventually become the bottleneck of the application.

In [15]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = list(csv.reader(f))
            self.header = read_file[0]
            self.rows = read_file[1:]
        for row in self.rows:    
            row[-1] = int(row[-1])    
        self.id_to_row = dict()   #Adding a level of preprocessing to make the lookup process faster.    
        for row in self.rows:
            self.id_to_row[row[0]] = row     #populating the dict()
                    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None            
            
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
            
            
laptops = Inventory('laptops.csv')
print(laptops.get_laptop_from_id_fast('3362737'))
print(laptops.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


### Time it!
Let's compare the times and see if the extra memory spent on the `dict()` has been worth it.

In [5]:
import time
import random

ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

laptops = Inventory('laptops.csv')

total_time_no_dict = 0
total_time_dict = 0

for i in ids:
    start = time.time()
    laptops.get_laptop_from_id(i)
    end = time.time()
    total_time_no_dict += end-start
    
for i in ids:
    start = time.time()
    laptops.get_laptop_from_id_fast(i)
    end = time.time()
    total_time_dict += end-start
    
print(total_time_no_dict)
print(total_time_dict)

1.697291612625122
0.006418466567993164


### Double trouble
The results have been quite interesting because there is a slight difference in how the methods can be written. Please consider the alternate answer below.

In [6]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = list(csv.reader(f))
            self.header = read_file[0]
            self.rows = read_file[1:]
        for row in self.rows:    
            row[-1] = int(row[-1])    
        self.id_to_row = dict()    
        for row in self.rows:
            self.id_to_row[row[0]] = row    
                    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
            else:                     #The only thing added here is an else: statement instead of returning None at the end.
                return None            
            
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        else:
            return None         #Here we also inserted an else statement.
        
#So when we time it again:
#It gives some unexpected results!
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

laptops = Inventory('laptops.csv')

total_time_no_dict = 0
total_time_dict = 0

for i in ids:
    start = time.time()
    laptops.get_laptop_from_id(i)
    end = time.time()
    total_time_no_dict += end-start
    
for i in ids:
    start = time.time()
    laptops.get_laptop_from_id_fast(i)
    end = time.time()
    total_time_dict += end-start
    
print(total_time_no_dict)
print(total_time_dict)

0.004761695861816406
0.011889934539794922


### What! Right?
They are both faster than the method above. And the strangest thing is, the method without a `dict()` is faster!
I am not able to explain the difference so I have sought some help. The answer will be added below when the time is right (pun intended ;-) One other thing that is plaguing the second method is that the results vary a lot. Sometimes one method is better sometimes the other. It is hard to actually compare the methods there. One thing is sure, they are about the same speed as the `no_dict` method from the first method.

**UPDATE** The method that I used with `else:` actually does not find the laptop at all. If I would have taken a look at the output of the function I would have noticed that. However the return statement immediately executes. So it is only able to find a laptop if it gets the first laptop as the input. At least there is one less mystery to solve.

## Gift card
Let's move on from this interesting example to another one:

Let's pretend we own a store and this is our lookup system. 

Sometimes, this fictional store offers a promotion where you give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

For example, imagine that there are only three laptops in inventory:

![image](https://dq-content.s3.amazonaws.com/481/laptop_promotion.png)

The prices of these laptops are \\$1,339, \\$898, and \\$575. Say we offered a gift card of \\$2,500. Since a customer can buy, at most, two laptops with a gift card, the maximum they can spend is \\$2,237 (\\$1,339 plus \\$898). Therefore, they might feel cheated because, no matter how they spend their gift card, they cannot spend the full \\$2,500.

We don't want to make a customer feel cheated, so whenever we issue a gift card, we want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for *D* dollars, you want to make sure that either there is a laptop that costs exactly *D* dollars or two laptops whose costs add up to precisely *D* dollars.

We will write a method to retrieve the options.

In [8]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = list(csv.reader(f))
            self.header = read_file[0]
            self.rows = read_file[1:]
            
        for row in self.rows:    
            row[-1] = int(row[-1])    
        self.id_to_row = dict()   
        
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None            #I have kept the first function. The decision is arbitrary :) 
            
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollar_amount):
        for row in self.rows:
            if row[-1] == dollar_amount: #maybe there is one laptop with the exact price of the voucher.
                return True
        for row in self.rows:
            for row2 in self.rows:
                if row + row2 == dollar_amount:
                    return True
        return False
    
laptops = Inventory('laptops.csv')
print(laptops.check_promotion_dollars(1000))        
print(laptops.check_promotion_dollars(442))  

True
False


### Preprocessing
Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.

In [10]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = list(csv.reader(f))
            self.header = read_file[0]
            self.rows = read_file[1:]
            
        for row in self.rows:    
            row[-1] = int(row[-1])
            
        self.id_to_row = dict()   
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        self.prices = set()     #The preprocessing is performed here.
        for row in self.rows:
            self.prices.add(row[-1])
            
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None  
            
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollar_amount):
        for row in self.rows:
            if row[-1] == dollar_amount: 
                return True
        for row in self.rows:
            for row2 in self.rows:
                if row + row2 == dollar_amount:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollar_amount):
        if dollar_amount in self.prices:
            return True
        for price in self.prices:
            if (dollar_amount - price) in self.prices:  #This is a clever way to see if two values add up
                return True
        return False
    
laptops = Inventory('laptops.csv')
print(laptops.check_promotion_dollars_fast(1000))        
print(laptops.check_promotion_dollars_fast(442))  

True
False


### Time it one more time!

In [13]:
laptops = Inventory('laptops.csv')

prices = [random.randint(100, 5000) for _ in range(100)]

total_time_no_set = 0
total_time_set = 0

for price in prices:
    start = time.time()
    laptops.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end-start
    
for price in prices:
    start = time.time()
    laptops.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end-start
    
print(total_time_no_set)
print(total_time_set)

61.02195882797241
0.0009963512420654297


As we can see, the method without preprocessing is taking a lot longer than the other one.

## Budget optimizing
At last we will optimize our laptops system to immediately find the laptops for a customer that is in their budget.
In order to do that we will use binary search on a sorted price list. Binary search uses a logarithmic time complexity because it cuts the list in half all the time to see if the value is above or below.

A bit like a guessing game where someone an only say above or below after a guessing a given value. 

In [14]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = list(csv.reader(f))
            self.header = read_file[0]
            self.rows = read_file[1:]
            
        for row in self.rows:    
            row[-1] = int(row[-1])
            
        self.id_to_row = dict()   
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        self.prices = set()    
        for row in self.rows:
            self.prices.add(row[-1])
            
        self.rows_by_price = sorted(self.rows, key = lambda x: x[-1])     #The preprocessing is performed here.
            
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None  
            
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollar_amount):
        for row in self.rows:
            if row[-1] == dollar_amount: 
                return True
        for row in self.rows:
            for row2 in self.rows:
                if row + row2 == dollar_amount:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollar_amount):
        if dollar_amount in self.prices:
            return True
        for price in self.prices:
            if (dollar_amount - price) in self.prices:  #This is a clever way to see if two values add up
                return True
        return False
    
    def find_first_laptop_more_expensive(self, target_price): #new method
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start

inventory = Inventory('laptops.csv')                                
print(inventory.find_first_laptop_more_expensive(1000))  
print(inventory.find_first_laptop_more_expensive(10000))

683
-1
